In [18]:

import numpy as np
import altair as alt
from tqdm import tqdm
import sys
sys.path.append('../../')
from Scanner.general_consts import *
# alt.data_transformers.enable('csv')
import datetime
# import deltatime
import plotly.graph_objects as go
import pandas as pd
import json
from pathlib import Path
import itertools
import csv
import json

import os

import pandas as pd
from scipy import integrate
import matplotlib.pyplot as plt
sys.path.insert(1, '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch')
import plotly.express as px
from resources.section_logtypes import section_logtypes
from resources.logtypes import logtypes
from experiment_manager import ExperimentManager
log_types_str = [f"{logtype[0].lower()}_{logtype[1]}" for logtype in logtypes]
# alt.renderers.enable('altair_saver', fmts=['png'])


In [6]:
env_names = []
dirs_names = os.listdir(f"/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__")
model_names = set()
reward_parameters = set()
learning_rates = set()
for dir in dirs_names:
    print(dir)
    if not dir.__contains__('splunk') or int(dir.split("splunk-v")[1]) <13:
        continue
    env_names.append(dir)
    for file in os.listdir(f"/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/{dir}"):

        if "__" not in file:
            continue
        info = file.split("__")
        model_name = info[0].split("_")[0]
        reward_parameter = '_'.join(info[0].split("_")[1:])
        lr = info[1]
        model_names.add(model_name)
        reward_parameters.add(reward_parameter)
        learning_rates.add(lr)

best_so_far
splunk-v7
splunk-v8
splunk-v3
splunk-v9
splunk-v10
splunk-v11
splunk-v12
last
splunk-v13
splunk-v14
splunk-v15


In [29]:
# Example usage
directory = '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__'  # Replace this with the path to the top-level directory
target_file = 'time_rules_energy.json'  # Specify the target file name to search for

In [38]:
import os
import json
import pandas as pd
import plotly.express as px

# Function to extract the splunk version from the prefix_path
def extract_splunk_version(prefix_path):
    return prefix_path.split('/')[1]  # Assuming the version is the second part of the path

# Function to find files and corresponding parameters in separate 'parameters_train.json' files
def find_files_and_params(directory, target_file=None, params_file='parameters_train.json'):
    results = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        if target_file in files:
            file_path = os.path.join(root, target_file)
            # Load the corresponding parameters file
            params_data = None
            if params_file in files:
                params_path = os.path.join(root, params_file)
                try:
                    with open(params_path, 'r') as f:
                        params_data = json.load(f)
                except Exception as e:
                    print(f"Error reading parameters file {params_path}: {e}")
            
            # Append the file path and params
            results.append({
                'file_path': file_path,
                'params': params_data
            })
    
    return results

# Function to extract data from the 'time_rules_energy.json' files
def extract_time_rules_data(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            return data
    except Exception as e:
        print(f"Error reading time rules file {file_path}: {e}")
        return None


# Find the target files and corresponding parameters
experiment_data = find_files_and_params(directory, target_file)

# Prepare data for plotting
for data in experiment_data:
    time_rules = extract_time_rules_data(data['file_path'])
    
    if time_rules:
        # Extract parameters from the parameters file
        params = data['params']
        if not params:
            print(f"No parameters found for {data['file_path']}")
            continue
        if 'prefix_path' not in params:
            print(f"Prefix path not found for {data['file_path']}")
            continue
        splunk_version = extract_splunk_version(params['prefix_path'])
        alpha = params.get('alpha', 'N/A')
        beta = params.get('beta', 'N/A')
        gamma = params.get('gamma', 'N/A')
        learning_rate = params.get('learning_rate', 'N/A')
        policy = params.get('policy', 'N/A')
        
        # Prepare data for plotting
        plot_data = []
        for entry in time_rules:
            time_range = eval(entry['time_range'])  # Convert time_range to tuple
            for key, value in entry.items():
                if key.startswith("rule_duration_"):
                    rule_name = key.replace("rule_duration_", "")
                    duration = value
                    alert_key = f"rule_alert_{rule_name}"
                    alert_count = entry.get(alert_key, 0)
                    
                    plot_data.append({
                        'start_time': time_range[0],
                        'end_time': time_range[1],
                        'rule_name': rule_name,
                        'duration': duration,
                        'alert_count': alert_count,
                    })
        
        # Convert plot_data to DataFrame
        df = pd.DataFrame(plot_data)
        if df.empty:
            print(f"No data found for {data['file_path']}")
            continue
        # Convert string time to datetime for better plotting
        df['start_time'] = pd.to_datetime(df['start_time'], format="%m/%d/%Y:%H:%M:%S")
        df['end_time'] = pd.to_datetime(df['end_time'], format="%m/%d/%Y:%H:%M:%S")
        
        # Create a Plotly Express plot
        title = f"Splunk Version: {splunk_version} | Alpha: {alpha}, Beta: {beta}, Gamma: {gamma}, Learning Rate: {learning_rate}, Policy: {policy}"
        fig = px.line(df, 
                      x='start_time', 
                      y='duration', 
                      color='rule_name', 
                      title=title,
                      labels={'duration': 'Duration', 'start_time': 'Time', 'rule_name': 'Rule Name'})
        
        # Show the plot
        fig.show()


No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/ppo_0.1667_0.1667_0.667/train/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/ppo_0.1667_0.1667_0.667/test/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/baseline_random/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/no_agent/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/a2c_0.1667_0.1667_0.667/train/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/best_so_far/splunk-v3/a2c_0.1667_0.1667_0.667/test/time_rules_energy.json
No paramete

No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v14/mlp_ppo_0.0_0.2_0.8__0.001/Experiment_3/test_75_05/28/2024:13:00:00/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v14/lstm_recurrentppo_0.0_0.2_0.8__0.001/Experiment_3/test_75_05/28/2024:13:00:00/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v14/lstm_recurrentppo_0.0_0.2_0.8__0.001/Experiment_4/test_75_05/28/2024:13:00:00/time_rules_energy.json
No parameters found for /home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v14/lstm_recurrentppo_0.0_0.2_0.8__0.001/Experiment_5/test_75_05/28/2024:13:00:00/time_rules_energy.json


In [25]:
df.params.tolist()


[{'alpha': 0.0,
  'beta': 0.2,
  'gamma': 0.8,
  'learning_rate': 0.001,
  'policy': 'mlp',
  'reward_calculator_version': '6',
  'ent_coef': 0.01,
  'df': 0.95,
  'prefix_path': 'experiments__/splunk-v14/mlp_ppo_0.0_0.2_0.8__0.001/Experiment_3'},
 {'alpha': 0.0,
  'beta': 0.2,
  'gamma': 0.8,
  'learning_rate': 0.001,
  'policy': 'mlp',
  'reward_calculator_version': '6',
  'ent_coef': 0.01,
  'df': 0.95,
  'prefix_path': 'experiments__/splunk-v14/mlp_ppo_0.0_0.2_0.8__0.001/Experiment_3'},
 {'alpha': 0.0,
  'beta': 0.2,
  'gamma': 0.8,
  'learning_rate': 0.001,
  'policy': 'mlp',
  'reward_calculator_version': '6',
  'ent_coef': 0.01,
  'df': 0.95,
  'prefix_path': 'experiments__/splunk-v14/mlp_ppo_0.0_0.2_0.8__0.001/Experiment_3'},
 {'alpha': 0.0,
  'beta': 0.2,
  'gamma': 0.8,
  'learning_rate': 0.001,
  'policy': 'mlp',
  'reward_calculator_version': '6',
  'ent_coef': 0.01,
  'df': 0.95,
  'prefix_path': 'experiments__/splunk-v14/mlp_ppo_0.0_0.2_0.8__0.001/Experiment_3'},
 {'alpha

In [13]:
# Main execution
num_of_steps = 60
figures = {}

base_dir = Path("/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__")
for env_name in env_names:
    all_results = {}
    for model_name, learning_rate, reward_parameter in itertools.product(model_names, learning_rates, reward_parameters):
        try:
            env_path = base_dir / env_name / f"{model_name}_{reward_parameter}__{learning_rate}/"
            experiment_paths = list(env_path.iterdir())
            print(experiment_paths)
            for experiment_path in experiment_paths:
                time_rules_energy_path = experiment_path / "time_rules_energy.json"
                # load the time_rules_energy file
                with open(time_rules_energy_path) as f:
                    time_rules_energy = json.load(f)
                df = pd.DataFrame(time_rules_energy)
                # Convert time_range to two columns 'start_time' and 'end_time'
                df['start_time'], df['end_time'] = zip(*df['time_range'].apply(lambda x: eval(x)))

                # Extract rule names and corresponding durations
                duration_cols = [col for col in df.columns if col.startswith('rule_duration_')]
                alert_cols = [col for col in df.columns if col.startswith('rule_alert_')]

                # Melt the dataframe to long format for easier plotting
                df_long = df.melt(id_vars=['start_time', 'end_time'], 
                                value_vars=duration_cols, 
                                var_name='rule', 
                                value_name='duration')

                # Remove 'rule_duration_' prefix from the rule names
                df_long['rule'] = df_long['rule'].str.replace('rule_duration_', '')

                # Create Plotly Express line plot
                fig = px.line(df_long, 
                            x='start_time', 
                            y='duration', 
                            color='rule', 
                            title='Rule Durations Over Time',
                            labels={'duration': 'Duration', 'start_time': 'Time', 'rule': 'Rule Name'})

                # Show the plot
                fig.show()
        except Exception as e:
            print(e)
            continue

[Errno 2] No such file or directory: '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/lstm_ppo_0.0_0.5_0.95__0.001'
[Errno 2] No such file or directory: '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/lstm_ppo_0.0_0.2_0.8__0.001'
[Errno 2] No such file or directory: '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/lstm_ppo_0.0_0.5_0.5__0.001'
[Errno 2] No such file or directory: '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/lstm_recurrentppo_0.0_0.2_0.8__0.001'
[PosixPath('/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/mlp_ppo_0.0_0.5_0.95__0.001/train')]
[Errno 2] No such file or directory: '/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experiments__/splunk-v13/mlp_ppo_0.0_0.5_0.95__0.001/train/time_rules_energy.json'
[PosixPath('/home/shouei/GreenSecurity-FirstExperiment/SplunkResearch/experimen